ETF Return Tracker
* May 16, 2025
* Version 5

Tutor:
* Anthropic's AI, Claude

This script:
* Analyzes ETF portfolio performance by calculating returns,
expenses, and net profit based on user inputs.
* Handles multiple ETFs and provide portfolio-level analysis. H


This script features:
1. Prompts the user for the ETF name, expense ratio, number of shares, purchase price, purchase date, current price, and current date
2. Calculates days held between the dates
3. Computes the total return in dollars
4. Calculates the annualized return percentage
5. Estimates the total expense cost based on the expense ratio and time held
6. Determines the net return (return minus expenses)
7. Displays all results in a readable format

Added error handling and data validation capabilities to the script

In [1]:
#!/usr/bin/env python3
"""
ETF Portfolio Analyzer

A Python script that analyzes a portfolio of ETFs by calculating returns,
expenses, and net profit for individual ETFs and the combined portfolio.

Features:
- Handles analysis of two different ETFs
- Prompts the user for ETF details (name, expense ratio, shares, prices, dates)
- Validates all user inputs with appropriate error handling
- Calculates days held between purchase and current date
- Computes total return in dollars
- Calculates annualized return percentage
- Estimates total expense cost based on the expense ratio and time held
- Determines the net return (return minus expenses)
- Provides portfolio-level analysis including asset allocation ratio
- Displays results in a readable format

Version: 5.0
Date: May 18, 2025
Author: Created with assistance from Anthropic's Claude
"""

import datetime
import sys
import re

def calculate_days_held(start_date_str, end_date_str):
    """
    Calculate the number of days between two dates.

    Args:
        start_date_str (str): The starting date in YYYY-MM-DD format
        end_date_str (str): The ending date in YYYY-MM-DD format

    Returns:
        int: Number of days between the two dates
    """
    # Convert string dates to datetime objects
    start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

    # Calculate days between dates
    days_held = (end_date - start_date).days
    return days_held

def calculate_annualized_return(start_price, end_price, days_held):
    """
    Calculate the annualized return percentage.

    Args:
        start_price (float): The initial price per share
        end_price (float): The current price per share
        days_held (int): Number of days the investment was held

    Returns:
        float: Annualized return as a percentage
    """
    if days_held <= 0:
        return 0

    # Calculate simple return
    simple_return = (end_price - start_price) / start_price

    # Convert to annualized return
    annualized_return = (1 + simple_return) ** (365 / days_held) - 1

    return annualized_return * 100  # Convert to percentage

def validate_etf_name(name):
    """
    Validate the ETF name.

    Args:
        name (str): The ETF name to validate

    Returns:
        str: Validated ETF name

    Raises:
        ValueError: If the name is invalid
    """
    if not name:
        raise ValueError("ETF name cannot be empty.")

    # Remove any whitespace and convert to uppercase
    name = name.strip().upper()

    # Check if the name consists of valid characters (letters, numbers, dots)
    if not re.match(r'^[A-Z0-9\.]+$', name):
        raise ValueError("ETF name should only contain letters, numbers, and dots.")

    return name

def validate_expense_ratio(ratio_str):
    """
    Validate and convert the expense ratio input.

    Args:
        ratio_str (str): The expense ratio as a string

    Returns:
        float: Validated expense ratio as a float

    Raises:
        ValueError: If the ratio is invalid
    """
    try:
        ratio = float(ratio_str)
    except ValueError:
        raise ValueError("Expense ratio must be a number.")

    if ratio < 0:
        raise ValueError("Expense ratio cannot be negative.")
    if ratio > 10:
        raise ValueError("Expense ratio seems too high (>10%). Please enter as a percentage (e.g., 0.05 for 0.05%).")

    return ratio

def validate_shares(shares_str):
    """
    Validate and convert the number of shares input.

    Args:
        shares_str (str): The number of shares as a string

    Returns:
        float: Validated number of shares as a float

    Raises:
        ValueError: If the shares value is invalid
    """
    try:
        shares = float(shares_str)
    except ValueError:
        raise ValueError("Number of shares must be a number.")

    if shares <= 0:
        raise ValueError("Number of shares must be greater than zero.")

    return shares

def validate_price(price_str):
    """
    Validate and convert the price input.

    Args:
        price_str (str): The price as a string

    Returns:
        float: Validated price as a float

    Raises:
        ValueError: If the price is invalid
    """
    try:
        price = float(price_str)
    except ValueError:
        raise ValueError("Price must be a number.")

    if price <= 0:
        raise ValueError("Price must be greater than zero.")

    return price

def validate_date(date_str):
    """
    Validate and convert the date input.

    Args:
        date_str (str): The date as a string in YYYY-MM-DD format

    Returns:
        str: Validated date string

    Raises:
        ValueError: If the date is invalid
    """
    # Check if the format is correct
    if not re.match(r'^\d{4}-\d{2}-\d{2}$', date_str):
        raise ValueError("Date must be in YYYY-MM-DD format.")

    try:
        # Attempt to convert to a datetime object to validate
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")

        # Check if date is in the future
        if date_obj > datetime.datetime.now():
            raise ValueError("Date cannot be in the future.")

        # Check if date is too far in the past (over 100 years)
        if date_obj < datetime.datetime.now() - datetime.timedelta(days=36500):
            raise ValueError("Date seems too old (>100 years ago).")

    except ValueError as e:
        if "unconverted data remains" in str(e) or "does not match format" in str(e):
            raise ValueError("Date must be in YYYY-MM-DD format.")
        else:
            raise e

    return date_str

def get_input_with_validation(prompt, validator, error_prefix=""):
    """
    Get user input with validation.

    Args:
        prompt (str): The prompt to display to the user
        validator (function): The validation function to use
        error_prefix (str): A prefix to add to error messages

    Returns:
        object: Validated input
    """
    while True:
        try:
            user_input = input(prompt)
            return validator(user_input)
        except ValueError as e:
            print(f"{error_prefix}Error: {e}")
            print("Please try again.")

def get_etf_data(etf_number):
    """
    Prompt the user for ETF details with input validation.

    Args:
        etf_number (int): The ETF number (1 or 2)

    Returns:
        dict: ETF data including name, expense ratio, shares, prices, and dates
    """
    print(f"\nEnter details for ETF #{etf_number}:")
    print("-" * 30)

    prefix = f"ETF #{etf_number}: "
    etf_data = {}

    # Get validated ETF name
    etf_data["name"] = get_input_with_validation(
        f"Enter ETF #{etf_number} name (e.g., VTI, BND): ",
        validate_etf_name,
        prefix
    )

    # Get validated expense ratio
    etf_data["expense_ratio"] = get_input_with_validation(
        f"Enter expense ratio for {etf_data['name']} (as percentage, e.g., 0.05 for 0.05%): ",
        validate_expense_ratio,
        prefix
    )

    # Get validated number of shares
    etf_data["shares"] = get_input_with_validation(
        f"Enter number of shares for {etf_data['name']}: ",
        validate_shares,
        prefix
    )

    # Get validated purchase price
    etf_data["start_price"] = get_input_with_validation(
        f"Enter price per share on purchase date for {etf_data['name']} ($): ",
        validate_price,
        prefix
    )

    # Get validated purchase date
    etf_data["start_date"] = get_input_with_validation(
        f"Enter purchase date for {etf_data['name']} (YYYY-MM-DD): ",
        validate_date,
        prefix
    )

    # Get validated current price
    etf_data["end_price"] = get_input_with_validation(
        f"Enter current price per share for {etf_data['name']} ($): ",
        validate_price,
        prefix
    )

    # Get validated current date
    etf_data["end_date"] = get_input_with_validation(
        f"Enter current date for {etf_data['name']} (YYYY-MM-DD): ",
        validate_date,
        prefix
    )

    # Validate that end date is after start date
    try:
        start_date = datetime.datetime.strptime(etf_data["start_date"], "%Y-%m-%d")
        end_date = datetime.datetime.strptime(etf_data["end_date"], "%Y-%m-%d")

        if end_date < start_date:
            raise ValueError("Current date must be after purchase date.")
    except ValueError as e:
        print(f"{prefix}Error: {e}")
        # Recursively call this function to get valid dates
        return get_etf_data(etf_number)

    return etf_data

def analyze_etf(etf_data):
    """
    Analyze a single ETF based on provided data.

    Args:
        etf_data (dict): ETF data including name, expense ratio, shares, prices, and dates

    Returns:
        dict: Analysis results including returns, expenses, and performance metrics
    """
    results = {}
    results["name"] = etf_data["name"]

    try:
        # Convert expense ratio from percentage to decimal
        expense_ratio_decimal = etf_data["expense_ratio"] / 100

        # Calculate days held
        days_held = calculate_days_held(etf_data["start_date"], etf_data["end_date"])
        results["days_held"] = days_held

        # Calculate returns
        initial_investment = etf_data["shares"] * etf_data["start_price"]
        current_value = etf_data["shares"] * etf_data["end_price"]
        total_return_dollars = current_value - initial_investment

        results["initial_investment"] = initial_investment
        results["current_value"] = current_value
        results["total_return_dollars"] = total_return_dollars

        # Calculate annualized return percentage
        annualized_return_pct = calculate_annualized_return(etf_data["start_price"], etf_data["end_price"], days_held)
        results["annualized_return_pct"] = annualized_return_pct

        # Calculate expenses
        years_held = days_held / 365
        total_expense_dollars = initial_investment * expense_ratio_decimal * years_held
        results["total_expense_dollars"] = total_expense_dollars

        # Calculate return less expenses
        net_return = total_return_dollars - total_expense_dollars
        results["net_return"] = net_return

    except Exception as e:
        print(f"Error analyzing ETF {etf_data['name']}: {e}")
        sys.exit(1)

    return results

def display_etf_results(results):
    """
    Display analysis results for a single ETF.

    Args:
        results (dict): Analysis results for the ETF
    """
    try:
        print(f"\nResults for {results['name']}:")
        print("-" * 30)
        print(f"Days Held: {results['days_held']}")
        print(f"Initial Investment: ${results['initial_investment']:.2f}")
        print(f"Current Value: ${results['current_value']:.2f}")
        print(f"Annualized Return: {results['annualized_return_pct']:.2f}%")
        print(f"Total Return: ${results['total_return_dollars']:.2f}")
        print(f"Total Expense Cost: ${results['total_expense_dollars']:.2f}")
        print(f"Return Less Expenses: ${results['net_return']:.2f}")
    except KeyError as e:
        print(f"Error displaying results: Missing data {e}")
        sys.exit(1)
    except Exception as e:
        print(f"Error displaying results: {e}")
        sys.exit(1)

def analyze_portfolio(etf1_results, etf2_results):
    """
    Analyze the combined portfolio of two ETFs.

    Args:
        etf1_results (dict): Analysis results for the first ETF
        etf2_results (dict): Analysis results for the second ETF

    Returns:
        dict: Portfolio-level analysis including combined metrics and allocation ratio
    """
    portfolio = {}

    try:
        # Calculate combined metrics
        portfolio["initial_investment"] = etf1_results["initial_investment"] + etf2_results["initial_investment"]
        portfolio["current_value"] = etf1_results["current_value"] + etf2_results["current_value"]
        portfolio["total_return_dollars"] = etf1_results["total_return_dollars"] + etf2_results["total_return_dollars"]
        portfolio["total_expense_dollars"] = etf1_results["total_expense_dollars"] + etf2_results["total_expense_dollars"]
        portfolio["net_return"] = etf1_results["net_return"] + etf2_results["net_return"]

        # Calculate weighted annualized return
        if portfolio["initial_investment"] > 0:
            etf1_weight = etf1_results["initial_investment"] / portfolio["initial_investment"]
            etf2_weight = etf2_results["initial_investment"] / portfolio["initial_investment"]
            portfolio["annualized_return_pct"] = (etf1_results["annualized_return_pct"] * etf1_weight +
                                                etf2_results["annualized_return_pct"] * etf2_weight)
        else:
            portfolio["annualized_return_pct"] = 0

        # Calculate portfolio allocation ratio
        if portfolio["current_value"] > 0:
            etf1_current_percent = (etf1_results["current_value"] / portfolio["current_value"]) * 100
            etf2_current_percent = (etf2_results["current_value"] / portfolio["current_value"]) * 100
            portfolio["allocation_ratio"] = f"{etf1_current_percent:.1f}/{etf2_current_percent:.1f}"
        else:
            portfolio["allocation_ratio"] = "0/0"

        portfolio["etf1_name"] = etf1_results["name"]
        portfolio["etf2_name"] = etf2_results["name"]

    except KeyError as e:
        print(f"Error analyzing portfolio: Missing data {e}")
        sys.exit(1)
    except Exception as e:
        print(f"Error analyzing portfolio: {e}")
        sys.exit(1)

    return portfolio

def display_portfolio_results(portfolio):
    """
    Display analysis results for the combined portfolio.

    Args:
        portfolio (dict): Portfolio-level analysis results
    """
    try:
        print("\nCombined Portfolio Results:")
        print("=" * 40)
        print(f"Total Initial Investment: ${portfolio['initial_investment']:.2f}")
        print(f"Total Current Value: ${portfolio['current_value']:.2f}")
        print(f"Portfolio Allocation: {portfolio['etf1_name']}/{portfolio['etf2_name']} = {portfolio['allocation_ratio']}")
        print(f"Weighted Annualized Return: {portfolio['annualized_return_pct']:.2f}%")
        print(f"Total Return: ${portfolio['total_return_dollars']:.2f}")
        print(f"Total Expense Cost: ${portfolio['total_expense_dollars']:.2f}")
        print(f"Return Less Expenses: ${portfolio['net_return']:.2f}")
    except KeyError as e:
        print(f"Error displaying portfolio results: Missing data {e}")
        sys.exit(1)
    except Exception as e:
        print(f"Error displaying portfolio results: {e}")
        sys.exit(1)

def main():
    """
    Main function to run the ETF Portfolio Analyzer.
    """
    try:
        print("ETF Portfolio Analyzer")
        print("=====================")
        print("This tool will analyze two ETFs and provide combined portfolio analysis.")

        # Get data for both ETFs
        etf1_data = get_etf_data(1)
        etf2_data = get_etf_data(2)

        # Analyze each ETF
        etf1_results = analyze_etf(etf1_data)
        etf2_results = analyze_etf(etf2_data)

        # Display individual ETF results
        display_etf_results(etf1_results)
        display_etf_results(etf2_results)

        # Analyze and display portfolio results
        portfolio = analyze_portfolio(etf1_results, etf2_results)
        display_portfolio_results(portfolio)

        print("\nAnalysis completed successfully!")

    except KeyboardInterrupt:
        print("\n\nProgram terminated by user.")
        sys.exit(0)
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

ETF Portfolio Analyzer
This tool will analyze two ETFs and provide combined portfolio analysis.

Enter details for ETF #1:
------------------------------
Enter ETF #1 name (e.g., VTI, BND): VTI
Enter expense ratio for VTI (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter number of shares for VTI: 10
Enter price per share on purchase date for VTI ($): 206.55
Enter purchase date for VTI (YYYY-MM-DD): 2022-05-31
Enter current price per share for VTI ($): 292.58
Enter current date for VTI (YYYY-MM-DD): 2025-05-18

Enter details for ETF #2:
------------------------------
Enter ETF #2 name (e.g., VTI, BND): BND
Enter expense ratio for BND (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter number of shares for BND: 3
Enter price per share on purchase date for BND ($): 76.55
Enter purchase date for BND (YYYY-MM-DD): 2022-05-31
Enter current price per share for BND ($): 72.48
Enter current date for BND (YYYY-MM-DD): 2025-05-18

Results for VTI:
------------------------------
Days Held: 1083
Init